In [1]:
import pandas as pd
import os

In [2]:
csv_path_list = [os.getcwd() + '/news_data.csv',
                 os.getcwd() + '/news_data2.csv']

df_list = list()
for csv_path in csv_path_list:
    print(f'{csv_path} start!')
    df = pd.read_table(csv_path, sep=',')

    df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    print(df.isnull().sum())
    
    print(f'중복제거 전 뉴스 기사 수 : {len(df)}')
    df.drop_duplicates(subset=['news'], inplace=True)
    print(f'중복제거 후 뉴스 기사 수 : {len(df)}')
    print(df.groupby('code').size().reset_index(name = 'count'), end='\n\n')
    
    df_list.append(df)

/home/aiffel0049/aiffel/Exploration/3.crawling/news_data.csv start!
news    0
code    0
dtype: int64
중복제거 전 뉴스 기사 수 : 5124
중복제거 후 뉴스 기사 수 : 3994
    code  count
0  IT/과학    903
1     사회   1668
2  생활/문화   1423

/home/aiffel0049/aiffel/Exploration/3.crawling/news_data2.csv start!
news    0
code    0
dtype: int64
중복제거 전 뉴스 기사 수 : 3703
중복제거 후 뉴스 기사 수 : 2137
    code  count
0  IT/과학    235
1     경제    902
2     사회    554
3  생활/문화    446



### 두 dataframe 합치기
https://rfriend.tistory.com/256

In [3]:
df_full = pd.concat(df_list, axis=0)
print(len(df_full))
print(df_full.groupby('code').size().reset_index(name = 'count'))

6131
    code  count
0  IT/과학   1138
1     경제    902
2     사회   2222
3  생활/문화   1869


### 형태소 추출

jvm에러는 java 설치   
https://www.digitalocean.com/community/tutorials/how-to-install-java-with-apt-on-ubuntu-18-04

In [4]:
from konlpy.tag import Mecab, Hannanum, Kkma, Komoran, Okt

mec_token = Mecab()
han_token = Hannanum()
kkm_token = Kkma()
kom_token = Komoran()
okt_token = Okt()
token_list = {'Mecab': mec_token, 
              'Hannanum': han_token, 
              'Kkma': kkm_token,
              'Komoran': kom_token,
              'Okt': okt_token}

### 불용어 제거

In [5]:
stopwords = ['에','는','은','을','했',
             '에게','있','이','의','하',
             '한','다','과','때문','할',
             '수','무단','따른','및','금지',
             '전재','경향신문','기자','는데','가',
             '등','들','파이낸셜','저작','등','뉴스']

In [6]:
def preprocessing(token: [Mecab, Hannanum, Kkma, Komoran, Okt], data):
    text_data = list()
    
    for sentence in data:
        temp_data = token.morphs(sentence)
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)
    
    text_data = list(map(' '.join, text_data))
    
    return text_data

In [9]:
import time
import pickle

if not os.path.isdir('preprocessing_data'):
    os.mkdir('preprocessing_data')

text_data_list = dict()
preprocessing_time = dict()
for token_name, token in token_list.items():
    preprocessing_data_path = os.getcwd() + f'/preprocessing_data/{token_name}_data.pkl'
    
    if os.path.exists(preprocessing_data_path):
        text_data = pickle.load(open(preprocessing_data_path, 'rb'))
    else:
        print(f'{token_name} preprocessing start')
        start_time = time.time()
        
        text_data = preprocessing(token, df_full['news'])
        
        pickle.dump(text_data, open(preprocessing_data_path, 'wb'))

        end_time = time.time() - start_time
        preprocessing_time[token_name] = end_time
        print('preprocessing time : ', end_time)
    
    text_data_list[token_name] = text_data
    print(len(text_data))

import json
json_path = os.getcwd() + '/preprocessing_data/preprocessing_time.json'
if not os.path.exists(json_path):
    json.dump(preprocessing_time, open(json_path, 'wt'), indent=4)
else:
    preprocessing_time = json.load(open(json_path, 'rt'))

6131
6131
6131
6131
6131


Mecab preprocessing start
preprocessing time :  7.921342372894287

Hannanum preprocessing start
preprocessing time :  210.64576864242554

Kkma preprocessing start
preprocessing time :  1656.435776233673

Komoran preprocessing start
preprocessing time :  89.45873713493347

Okt preprocessing start
preprocessing time :  197.1936264038086

In [10]:
from collections import OrderedDict
def show_count_morphs(text_data_list):
    morphs_count = dict()
    for token_name, data_list in text_data_list.items():
        print(f"{token_name}'s data counting start")
        for data in data_list:
            morphs = data.split()
            for morph in morphs:
                if morph not in morphs_count:
                    morphs_count[morph] = 1
                else:
                    morphs_count[morph] += 1
    
    reverse_sorted_list = OrderedDict(sorted(morphs_count.items(), key=lambda t: t[1], reverse=True))
    sorted_list = OrderedDict(sorted(morphs_count.items(), key=lambda t: t[1]))
    return reverse_sorted_list, sorted_list

In [11]:
many_count_list, few_count_list = show_count_morphs(text_data_list)

many_count_stopwords = list(many_count_list)[:20]
few_count_stopwords = list(few_count_list)[:20]

print(many_count_stopwords)
print(few_count_stopwords)

Mecab's data counting start
Hannanum's data counting start
Kkma's data counting start
Komoran's data counting start
Okt's data counting start
['ㄴ', '를', '고', '으로', '어', '에서', '로', '되', '도', '었', 'ㄹ', '일', '것', '아', '와', '지', '기', '았', '적', '게']
['코로나다', '어지러운', '스트로스', '박혔', '이용균', '참교', '설대우', '왕자서', '마운트배튼', '광천', '전태', '향해야', '모군', '파흐', '다고들', '박홍두', '새빨개', '달집', '시윤', '정창원']


In [12]:
def extra_preprocessing_by_new_stopwords(text_data_list):
    text_data_list_many = dict()
    text_data_list_few = dict()
    for token_name, text_data in text_data_list.items():
        print(f'{token_name} extra preprocessing start')
        many_count_preprocessed = list()
        few_count_preprocessed = list()
        for news_text in text_data:
            many_news_text = list()
            few_news_text = list()
            
            morphs = news_text.split()
            for morph in morphs:
                if morph not in many_count_stopwords:
                    many_news_text.append(morph)
                if morph not in few_count_stopwords:
                    few_news_text.append(morph)
            many_count_preprocessed.append(many_news_text)
            few_count_preprocessed.append(few_news_text)
        
        many_count_preprocessed = list(map(' '.join, many_count_preprocessed))
        few_count_preprocessed = list(map(' '.join, few_count_preprocessed))
        text_data_list_many[token_name] = many_count_preprocessed
        text_data_list_few[token_name] = few_count_preprocessed
    
    return text_data_list_many, text_data_list_few

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

def train_test(text_data_list):
    def tfidf_vectorizer(data):
        data_counts = count_vect.transform(data)
        data_tfidf = tfidf_transformer.transform(data_counts)
        return data_tfidf

    for token_name, text_data in text_data_list.items():
        #- 훈련 데이터와 테스트 데이터를 분리합니다.
        X_train, X_test, y_train, y_test = train_test_split(text_data, df_full['code'], random_state = 0)

        print(token_name, ' start')
        print('훈련용 뉴스 기사의 개수 :', len(X_train))
        print('테스트용 뉴스 기사의 개수 : ', len(X_test))
        print('훈련용 레이블의 개수 : ', len(y_train))
        print('테스트용 레이블의 개수 : ', len(y_test))

        #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
        count_vect = CountVectorizer()
        X_train_counts = count_vect.fit_transform(X_train)

        #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
        tfidf_transformer = TfidfTransformer()
        X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

        #- 나이브 베이즈 분류기를 수행합니다.
        #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
        clf = MultinomialNB().fit(X_train_tfidf, y_train) 

        y_pred = clf.predict(tfidf_vectorizer(X_test))
        print(metrics.classification_report(y_test, y_pred))

text_data_list_many, text_data_list_few = extra_preprocessing_by_new_stopwords(text_data_list)

Mecab extra preprocessing start
Hannanum extra preprocessing start
Kkma extra preprocessing start
Komoran extra preprocessing start
Okt extra preprocessing start


In [20]:
train_test(text_data_list)

Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.88      0.45      0.59       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.76      0.77       473

    accuracy                           0.77      1533
   macro avg       0.81      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.67      0.75       279
          경제       0.89      0.40      0.55       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0

In [17]:
train_test(text_data_list_many)

Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.83      0.75      0.79       279
          경제       0.88      0.45      0.60       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.68      0.75       279
          경제       0.88      0.41      0.56       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0

In [18]:
train_test(text_data_list_few)

Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.88      0.45      0.59       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.76      0.77       473

    accuracy                           0.77      1533
   macro avg       0.81      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.67      0.75       279
          경제       0.89      0.40      0.55       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0

###  불용어가 30개인 경우

<pre>
Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.88      0.45      0.59       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.76      0.77       473

    accuracy                           0.77      1533
   macro avg       0.81      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.67      0.75       279
          경제       0.89      0.40      0.55       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0.71      1533
weighted avg       0.76      0.74      0.73      1533

Kkma  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.83      0.73      0.78       279
          경제       0.86      0.47      0.60       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Komoran  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.87      0.45      0.59       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Okt  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.69      0.76       279
          경제       0.85      0.41      0.55       234
          사회       0.69      0.92      0.79       547
       생활/문화       0.76      0.75      0.76       473

    accuracy                           0.75      1533
   macro avg       0.79      0.69      0.71      1533
weighted avg       0.77      0.75      0.74      1533
</pre

### 불용어가 50개면서, 가장 카운트 많은 형태소를 불용어로 설정

<pre>
Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.83      0.75      0.79       279
          경제       0.88      0.45      0.60       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.68      0.75       279
          경제       0.88      0.41      0.56       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0.71      1533
weighted avg       0.76      0.74      0.73      1533

Kkma  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.83      0.74      0.78       279
          경제       0.86      0.47      0.61       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.77      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Komoran  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.87      0.45      0.59       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Okt  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.69      0.76       279
          경제       0.85      0.42      0.56       234
          사회       0.70      0.92      0.79       547
       생활/문화       0.76      0.75      0.75       473

    accuracy                           0.75      1533
   macro avg       0.79      0.69      0.72      1533
weighted avg       0.77      0.75      0.74      1533
<pre>

### 불용어가 50개면서, 가장 카운트 적은 형태소를 불용어로 설정

<pre>
Mecab  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.88      0.45      0.59       234
          사회       0.72      0.92      0.81       547
       생활/문화       0.78      0.76      0.77       473

    accuracy                           0.77      1533
   macro avg       0.81      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Hannanum  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.67      0.75       279
          경제       0.89      0.40      0.55       234
          사회       0.70      0.91      0.79       547
       생활/문화       0.72      0.76      0.74       473

    accuracy                           0.74      1533
   macro avg       0.79      0.69      0.71      1533
weighted avg       0.76      0.74      0.73      1533

Kkma  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.83      0.73      0.78       279
          경제       0.86      0.47      0.60       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Komoran  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.84      0.75      0.79       279
          경제       0.87      0.45      0.59       234
          사회       0.73      0.92      0.81       547
       생활/문화       0.78      0.77      0.77       473

    accuracy                           0.77      1533
   macro avg       0.80      0.72      0.74      1533
weighted avg       0.78      0.77      0.76      1533

Okt  start
훈련용 뉴스 기사의 개수 : 4598
테스트용 뉴스 기사의 개수 :  1533
훈련용 레이블의 개수 :  4598
테스트용 레이블의 개수 :  1533
              precision    recall  f1-score   support

       IT/과학       0.85      0.69      0.76       279
          경제       0.85      0.41      0.55       234
          사회       0.69      0.92      0.79       547
       생활/문화       0.76      0.75      0.76       473

    accuracy                           0.75      1533
   macro avg       0.79      0.69      0.71      1533
weighted avg       0.77      0.75      0.74      1533
</pre>

In [21]:
# TODO 세 케이스에 대한 분석 필요